# How to perform analysis?

This notebook contains a guide to perform analysis using GWKokab. We will recreate the analysis shared in [Eccentricity matters: Impact of eccentricity on inferred binary black hole populations](https://arxiv.org/abs/2404.08185). Data required for analysis can be generated by the [Synthetic Data Generation with GWKokab](https://github.com/gwkokab/gwkokab/blob/main/examples/synthetic_data.ipynb) tutorial.

In [ ]:
import os


os.environ["NPROC"] = "4"
os.environ["intra_op_parallelism_threads"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import jax


jax.config.update("jax_compilation_cache_dir", "cache-location")

In [ ]:
!wget -c https://raw.githubusercontent.com/gwkokab/asset-store/main/neural_vts/neural_vt_1_200_1000.eqx

In [ ]:
from glob import glob

import numpy as np
from jax import random as jrd, vmap
from numpyro import distributions as dist
from numpyro.distributions import HalfNormal

from gwkokab.inference import BayesianHierarchicalModel, flowMChandler, ModelPack
from gwkokab.models import Wysocki2019MassModel
from gwkokab.parameters import (
    ECCENTRICITY,
    Parameter,
    PRIMARY_MASS_SOURCE,
    SECONDARY_MASS_SOURCE,
)
from gwkokab.vts.neuralvt import load_model

To make our results reproducible, we will set the random seed to a fixed value.

In [ ]:
SEED = 37
KEY = jrd.PRNGKey(SEED)

N_DIM = 5
N_CHAINS = 20

We have five parameters to recover from the data: $\log{(\mathcal{R})}$, $\alpha_m$, $m_{\text{min}}$, $m_{\text{max}}$, and $\sigma$.

*Note: $\log(\mathcal{R})$ is in common logarithms. Set your priors accordingly*

In [ ]:
LOG_RATE = Parameter(name="log_rate", prior=dist.Uniform(low=-1, high=3))
ALPHA = Parameter(name="alpha_m", prior=dist.Uniform(low=-5.0, high=5.0))
MASS_MIN = Parameter(name="mmin", prior=dist.Uniform(low=5.0, high=20.0))
MASS_MAX = Parameter(name="mmax", prior=dist.Uniform(low=30.0, high=100.0))
SIGMA_ECC = Parameter(name="scale", prior=dist.Uniform(low=0.0, high=0.5))

We have to use `gwkokab.inference.ModelPack` to define a model.

- `name`: Uninitialized `numpyro.distributions.Distribution` object.
- `output`: Outputs of the model.
- `parameters_to_recover`: list of `Parameter` objects to recover.
- `arguments`: Additional arguments to the model or value of the fixed parameters.

Although $\log(\mathcal{R})$ has no model, we define it as a model with `name=None` and `output=[]`. Using this [judaad](https://en.wikipedia.org/wiki/Jugaad) we do not have modify the `BayesianHierarchicalModel`.

In [ ]:
log_rate_model = ModelPack(name=None, output=[], parameters_to_recover=[LOG_RATE])

m1m2_model = ModelPack(
    name=Wysocki2019MassModel,
    output=[PRIMARY_MASS_SOURCE(), SECONDARY_MASS_SOURCE()],
    parameters_to_recover=[ALPHA, MASS_MIN, MASS_MAX],
)

ecc_model = ModelPack(
    name=HalfNormal,
    output=[ECCENTRICITY()],
    parameters_to_recover=[SIGMA_ECC],
    arguments={"validate_args": True},
)

Define paths to the necessary files.

In [ ]:
POSTERIOR_REGEX = os.getcwd() + r"/data/realization_0/posteriors/event_*.dat"
VT_FILENAME = os.getcwd() + r"/neural_vt_1_200_1000.eqx"
VT_PARAMS = [PRIMARY_MASS_SOURCE().name, SECONDARY_MASS_SOURCE().name]
ANALYSIS_TIME = 248

print(POSTERIOR_REGEX)
print(VT_FILENAME)

In [ ]:
posteriors = glob(POSTERIOR_REGEX)
data_set = {
    "data": [np.loadtxt(event) for event in posteriors],
    "N": len(posteriors),
}

In [ ]:
def get_logVT():
    _, logVT = load_model(VT_FILENAME)
    return vmap(logVT)

`flowMC` has variety of local samplers. To make workflow faster, flexible, and more efficient, we create a dictionary that contains a key `"sampler"` and a value of the sampler name, and rest of the arguments that sampler will accept, everything else will be managed by `flowMChandler`. This style of defining samplers is same for NF-model, Sampler and information required to store the meta data provided by the flowMC sampler.

In [ ]:
LOCAL_MALA_SAMPLER_KWARGS = {
    "sampler": "MALA",
    "step_size": 1e-2,
    "jit": True,
}

NF_MODEL_KWARGS = {
    "model": "MaskedCouplingRQSpline",
    "n_layers": 5,
    "hidden_size": [32, 32],
    "num_bins": 8,
    "n_features": N_DIM,
    "key": KEY,
}

_, KEY = jrd.split(KEY)

SAMPLER_KWARGS = {
    "n_dim": N_DIM,
    "rng_key": KEY,
    "data": None,
    "n_chains": N_CHAINS,
    "n_local_steps": 100,
    "n_global_steps": 65,
    "n_loop_training": 20,
    "n_loop_production": 20,
    "batch_size": 10000,
    "n_epochs": 5,
    "learning_rate": 0.001,
    "momentum": 0.9,
    "precompile": False,
    "verbose": False,
    "use_global": True,
    "logging": True,
    "outdir": "inf-plots",
    # "train_thinning":,
    # "output_thinning":,
    # "n_max_examples":,
    # "n_flow_sample":,
}

DATA_DUMP_KWARGS = {
    "out_dir": "sampler_data",
    "labels": ["alpha_m", "mmin", "mmax", "\sigma_ecc", "\log_rate"],
    "n_samples": 10000,
}

We use `BayesianHierarchicalModel` to get the posterior function. We have to pass model in order, and model for the rate should be the last one.

In [ ]:
lhbm = BayesianHierarchicalModel(
    m1m2_model,
    ecc_model,
    log_rate_model,
    vt_params=VT_PARAMS,
    logVT=get_logVT(),
    time=ANALYSIS_TIME,
)

Initializing the chains.

In [ ]:
_, KEY = jrd.split(KEY)

initial_position = lhbm.population_priors.sample(KEY, (N_CHAINS,))

Handler for flowMC setup.

In [ ]:
handler = flowMChandler(
    logpdf=lhbm.log_posterior,
    initial_position=initial_position,
    local_sampler_kwargs=LOCAL_MALA_SAMPLER_KWARGS,
    nf_model_kwargs=NF_MODEL_KWARGS,
    sampler_kwargs=SAMPLER_KWARGS,
    data_dump_kwargs=DATA_DUMP_KWARGS,
    data=data_set,
)

Run the sampler.

In [ ]:
handler.run()